In [4]:
import gspread
import pandas as pd
from fractions import Fraction
import re

# Pastikan permision access di menu "Share" di rubah ke anyone with the link terlebih dahulu, agar bisa mengunakan cara OAuth
# Jika Ingin lebih dibatasi menggunakan cara google service account lebih disarankan

gc = gspread.oauth(
    credentials_filename= r'..\..\Data\Google Credentials\karma-oauth.json'
)

SHEET_ID = '1k-70BJNRm01RE2S7gBXuOfqxofk1gYz4g6lPbLUCklw'
# SHEET_NAME = 'DB'
SHEET_NAME = 'DB Test'


In [5]:
# Open the spreadsheet
spreadsheet_source = gc.open_by_key(SHEET_ID)

# Get all sheet names
sheet_names = [sheet.title for sheet in spreadsheet_source.worksheets()]
sheet_names

['DB',
 'DB Test',
 'DV',
 'List Events Transition',
 'Template',
 'Summary Report',
 'Past Attedants',
 'PAST EVENTS - Summary Report',
 'HOME ALONE IN CONCERT 5 Dec',
 'ASCOT FESTIVE DAY AT THE RACES 20 Dec',
 'CHARITY GALA 11 Dec',
 'A FEAST OF FITNESS IN GOA 10-14 Dec',
 'AYURVEDIC CURATIVES 6-9 Dec',
 '12 DAYS OF KARMA BAVARIA',
 '12 DAYS OF KARMA KK',
 '12 DAYS OF KARMA KLOM',
 '12 DAYS OF KARMA KSH',
 'ANCIENT BREWS OF THE AEGEAN 23-26 Nov',
 'A VIP EXPERIENCE WITH RUGBY LEGENDS! 9 Nov',
 'URBAN POLO: HIGH SOCIETY 16 Nov',
 'SPRITZ INTO SUMMER 17 Nov',
 'MELBOURNE MEMBER MIXER! 3 Nov',
 'HEART & SOUL OF OKTOBERFEST 24-27 Sep',
 'TACKLE, TRAIN & TOAST 27-30 Sep',
 'AWAKEN YOUR RADIANCE WITH BALIS GLOW GURU 4-7 Oct',
 'CARCASSONNE GASTRONOMY 11-13 Oct',
 'LITERATURE LOVERS WEEKEND 11-13 Oct',
 'DUSSEHRA: HIMACHALI RITUALS 11-14 Oct',
 'ANDALUSIA FITNESS & FLOW WITH JO 14-18 Oct',
 'HIDE & FEAST:CHEF JOES EPICUREAN 18-19 Oct',
 'SURREALIST SOJOURN IN SOUTHERN SPAIN 24-28 Oct',
 'SE

In [6]:
# get data gsheet

spreadsheet = gc.open_by_key(SHEET_ID)
worksheet = spreadsheet.worksheet(SHEET_NAME)
rows = worksheet.get_all_records()

# print(rows[:5])

# print('==============================')
# df = pd.DataFrame(rows, dtype="string")
df = pd.DataFrame(rows)
# df['Mobile Number'] = df['Mobile Number'].apply(Fraction)
df

,No,Servicing Office,Membership Type,Member No,First Name,Last Name,Email,Tel/Phone,Status,Staff Name,...,Event Type,Event Status,Event Name,Event Sheet Link,Event Name Detail,Event Link,Available Spots,Quota Event,Point Used,Paid (Usd)
0,2,KCB,Karma Club,11111,Fajar,Fatoni,fajar.fatoni@karmagroup.com,61413884302,Confirmed,Sri,...,,Currently Working On,THE NILE'S VIP VOYAGE 13-20 FEB,https://docs.google.com/spreadsheets/d/1AfxaRO...,The Nile’s VIP Voyage,https://karmaclub.karmagroup.com/karma-curated...,-24,,,
1,6,KCI,Karma Club,,Benny,Risanto,benny.risanto@karmagroup.com,61451125755,Confirmed,Finosha,...,,Currently Working On,THE NILE'S VIP VOYAGE 13-20 FEB,https://docs.google.com/spreadsheets/d/1AfxaRO...,The Nile’s VIP Voyage,https://karmaclub.karmagroup.com/karma-curated...,-24,,,
2,8,KCI,Karma Royal Residences,33333,Adhitya,Utomo,adhitya.putra@karmagroup.com,61411844008,Confirmed,Sri,...,,Currently Working On,THE NILE'S VIP VOYAGE 13-20 FEB,https://docs.google.com/spreadsheets/d/1AfxaRO...,The Nile’s VIP Voyage,https://karmaclub.karmagroup.com/karma-curated...,-24,,,
3,9,KCB,Karma Club,44444,Novalita,Rahayu,novalita.rahayu@karmagroup.com,61418189870,Confirmed,Sri,...,,Currently Working On,THE NILE'S VIP VOYAGE 13-20 FEB,https://docs.google.com/spreadsheets/d/1AfxaRO...,The Nile’s VIP Voyage,https://karmaclub.karmagroup.com/karma-curated...,-24,,,
4,13,KCB,Karma Club,55555,Wayan,Wirka,wayan.wirka@karmagroup.com,61414969481,Confirmed,Sri,...,,Currently Working On,THE NILE'S VIP VOYAGE 13-20 FEB,https://docs.google.com/spreadsheets/d/1AfxaRO...,The Nile’s VIP Voyage,https://karmaclub.karmagroup.com/karma-curated...,-24,,,
5,13,KCB,Karma Club,55555,Wayan,Wirka,adhitya320@gmail.com,61414969481,Confirmed,Sri,...,,Currently Working On,THE NILE'S VIP VOYAGE 13-20 FEB,https://docs.google.com/spreadsheets/d/1AfxaRO...,The Nile’s VIP Voyage,https://karmaclub.karmagroup.com/karma-curated...,-24,,,


In [7]:
new_df = df.loc[(df['Event Name Detail'] != "") & 
                (df['Event Date From'] != "") & 
                (df['Event Date To'] != "")].copy()

In [8]:
# Convert 'Event Date From' and 'Event Date To' to datetime
new_df['Event Date From'] = pd.to_datetime(new_df['Event Date From'], errors='coerce')
new_df['Event Date To'] = pd.to_datetime(new_df['Event Date To'], errors='coerce')

# Create a new column for event name and feedback
def create_event_feedback(row):
    # Format the dates to "07 Apr"
    date_from = row['Event Date From'].strftime('%d %b')
    date_to = row['Event Date To'].strftime('%d %b')

    # Extract the year from 'Event Date To'
    year = row['Event Date To'].year
    
    if date_from == date_to:
        return f"{row['Event Name Detail']} ({date_from} {year})"
    else:
        return f"{row['Event Name Detail']} ({date_from} to {date_to} {year})"

new_df['Event Name Guest Feedback'] = new_df.apply(create_event_feedback, axis=1)
new_df

,No,Servicing Office,Membership Type,Member No,First Name,Last Name,Email,Tel/Phone,Status,Staff Name,...,Event Status,Event Name,Event Sheet Link,Event Name Detail,Event Link,Available Spots,Quota Event,Point Used,Paid (Usd),Event Name Guest Feedback
0,2,KCB,Karma Club,11111,Fajar,Fatoni,fajar.fatoni@karmagroup.com,61413884302,Confirmed,Sri,...,Currently Working On,THE NILE'S VIP VOYAGE 13-20 FEB,https://docs.google.com/spreadsheets/d/1AfxaRO...,The Nile’s VIP Voyage,https://karmaclub.karmagroup.com/karma-curated...,-24,,,,The Nile’s VIP Voyage (13 Feb to 20 Feb 2025)
1,6,KCI,Karma Club,,Benny,Risanto,benny.risanto@karmagroup.com,61451125755,Confirmed,Finosha,...,Currently Working On,THE NILE'S VIP VOYAGE 13-20 FEB,https://docs.google.com/spreadsheets/d/1AfxaRO...,The Nile’s VIP Voyage,https://karmaclub.karmagroup.com/karma-curated...,-24,,,,The Nile’s VIP Voyage (13 Feb to 20 Feb 2025)
2,8,KCI,Karma Royal Residences,33333,Adhitya,Utomo,adhitya.putra@karmagroup.com,61411844008,Confirmed,Sri,...,Currently Working On,THE NILE'S VIP VOYAGE 13-20 FEB,https://docs.google.com/spreadsheets/d/1AfxaRO...,The Nile’s VIP Voyage,https://karmaclub.karmagroup.com/karma-curated...,-24,,,,The Nile’s VIP Voyage (13 Feb to 20 Feb 2025)
3,9,KCB,Karma Club,44444,Novalita,Rahayu,novalita.rahayu@karmagroup.com,61418189870,Confirmed,Sri,...,Currently Working On,THE NILE'S VIP VOYAGE 13-20 FEB,https://docs.google.com/spreadsheets/d/1AfxaRO...,The Nile’s VIP Voyage,https://karmaclub.karmagroup.com/karma-curated...,-24,,,,The Nile’s VIP Voyage (13 Feb to 20 Feb 2025)
4,13,KCB,Karma Club,55555,Wayan,Wirka,wayan.wirka@karmagroup.com,61414969481,Confirmed,Sri,...,Currently Working On,THE NILE'S VIP VOYAGE 13-20 FEB,https://docs.google.com/spreadsheets/d/1AfxaRO...,The Nile’s VIP Voyage,https://karmaclub.karmagroup.com/karma-curated...,-24,,,,The Nile’s VIP Voyage (13 Feb to 20 Feb 2025)
5,13,KCB,Karma Club,55555,Wayan,Wirka,adhitya320@gmail.com,61414969481,Confirmed,Sri,...,Currently Working On,THE NILE'S VIP VOYAGE 13-20 FEB,https://docs.google.com/spreadsheets/d/1AfxaRO...,The Nile’s VIP Voyage,https://karmaclub.karmagroup.com/karma-curated...,-24,,,,The Nile’s VIP Voyage (13 Feb to 20 Feb 2025)


In [9]:
new_df.loc[new_df['Event Name Detail'].str.contains('Himalayan Skiing & Winter Carnival Adventure', regex=True, flags=re.I)]

,No,Servicing Office,Membership Type,Member No,First Name,Last Name,Email,Tel/Phone,Status,Staff Name,...,Event Status,Event Name,Event Sheet Link,Event Name Detail,Event Link,Available Spots,Quota Event,Point Used,Paid (Usd),Event Name Guest Feedback


In [10]:
new_df.to_excel(r"C:\Users\fajar.fatoni\Documents\Python\Data\newsletter_zoho.xlsx", index=False)